<h1>Moby Pick</h1>

This notebook uses the CSV files found in the following collection of datasets: https://mengtingwan.github.io/data/goodreads.html and converts them into a format that we can use in our Moby Pick book recommendation engine.

In [2]:
import sys
import os
import itertools
from operator import add
from csv import reader
from itertools import chain
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, expr, udf, collect_list, struct, array, lit, array_max, when
from pyspark.sql.types import FloatType, StringType, ArrayType, IntegerType, StructType, StructField, LongType

#from polyglot.detect import Detector
from langdetect import detect

In [3]:
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("Python Spark SQL basic example") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 19:41:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/12 19:41:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


First we import the 'works' dataset, which is "the abstract version of a book regardless any particular editions" per the dataset descriptions.

In [4]:
#path = "../goodreads_book_genres_initial.json"
path = "./Datasets/goodreads_book_works.json"
bookDF = spark.read.json(path)

best_book_id corresponds to the book ID in the reviews dataset, the book ID in the genres dataset, and the book ID in the books dataset so that's what we'll use as the primary identifier for a row.

In [5]:
#bookDF.printSchema()
#bookDF.show(10)
bookDF.filter(bookDF.best_book_id == 17310087).show()

+------------+-----------+--------------------------+---------------------------------+----------+--------------------+------------------------+--------------------------+-------------------------+--------------------+--------------------+-------------+-----------+-------------+------------------+--------+
|best_book_id|books_count|default_chaptering_book_id|default_description_language_code|media_type|original_language_id|original_publication_day|original_publication_month|original_publication_year|      original_title|         rating_dist|ratings_count|ratings_sum|reviews_count|text_reviews_count| work_id|
+------------+-----------+--------------------------+---------------------------------+----------+--------------------+------------------------+--------------------------+-------------------------+--------------------+--------------------+-------------+-----------+-------------+------------------+--------+
|    17310087|          1|                          |                       

First filter the data so we aren't getting anything that isn't books (e.g. the dataset includes audiobooks and some other media).

In [7]:
bookDF = bookDF.filter(col('original_title') != '').filter(col('media_type') == 'book')
#bookDF.count()

We only want the books that have gotten the most interactions / are the most popular. We order by the number of ratings the book has had as a proxy value for how popular a book is. Then we limit the dataset to 2000 books, since it's so large.

In [8]:
bookDF = bookDF.orderBy(bookDF.ratings_count.cast("int"), ascending=False).limit(2000)

Take a look and make sure we still have the most popular books in the dataset:

In [9]:
bookDF.filter(col("original_title").like("%Potter%")).select('best_book_id',\
            'original_publication_year',\
            'original_title',\
            'rating_dist',\
            'ratings_count',\
            'reviews_count',\
            'work_id'
           ).show()

+------------+-------------------------+--------------------+--------------------+-------------+-------------+--------+
|best_book_id|original_publication_year|      original_title|         rating_dist|ratings_count|reviews_count| work_id|
+------------+-------------------------+--------------------+--------------------+-------------+-------------+--------+
|           3|                     1997|Harry Potter and ...|5:3131920|4:11907...|      4972886|      5801988| 4640799|
|           5|                     1999|Harry Potter and ...|5:1303937|4:51915...|      2019176|      2496720| 2402163|
|       15881|                     1998|Harry Potter and ...|5:1097387|4:56060...|      1955144|      2431183| 6231171|
|           6|                     2000|Harry Potter and ...|5:1227707|4:50391...|      1912948|      2375359| 3046572|
|      136251|                     2007|Harry Potter and ...|5:1351479|4:39082...|      1889600|      2383409| 2963218|
|           2|                     2003|

Get the average rating in addition to the distribution, count, and sum.

In [10]:
bookDF = bookDF.select('best_book_id',\
            'original_publication_day',\
            'original_publication_month',\
            'original_publication_year',\
            'original_title',\
            'rating_dist',\
            'ratings_count',\
            'ratings_sum',\
            'reviews_count',\
            'text_reviews_count',\
            'work_id'
           ).withColumn("avg_rating", col("ratings_sum") / col("ratings_count"))

<h4>Language</h4>

None of the rows in the dataset have an entry for `default_description_language_code`. Because we need language in order to create recommendations for users, we need to fill in a language somehow. We create a UDF to detect the language from the title and use a python package called `langdetect`. There's no need to check the language in the dataset before filling, because again they're all null.

In [11]:
def get_language(title):
    try:
        inf_lang = detect(title)
        return inf_lang
    except Exception as e:
        return ''

lang_udf = udf(get_language, StringType())

In [12]:
bookDFLanguages = bookDF.withColumn('inferred_language_id', lang_udf(bookDF.original_title))

<h4>Genre</h4>

Genre is in a separate CSV, so we import it as another dataframe. The genre CSV has a struct type for the genres field. We'll do processing to select the "best-fit" genre, indicated by the key in the struct with the highest value.

In [14]:
# join in the genres information
pathG = "./Datasets/goodreads_book_genres_initial.json"
genreDF = spark.read.json(pathG)

In [15]:
genreDF.printSchema()
genreDF.show(5, truncate=False)

root
 |-- book_id: string (nullable = true)
 |-- genres: struct (nullable = true)
 |    |-- children: long (nullable = true)
 |    |-- comics, graphic: long (nullable = true)
 |    |-- fantasy, paranormal: long (nullable = true)
 |    |-- fiction: long (nullable = true)
 |    |-- history, historical fiction, biography: long (nullable = true)
 |    |-- mystery, thriller, crime: long (nullable = true)
 |    |-- non-fiction: long (nullable = true)
 |    |-- poetry: long (nullable = true)
 |    |-- romance: long (nullable = true)
 |    |-- young-adult: long (nullable = true)

+-------+---------------------------------------------------------+
|book_id|genres                                                   |
+-------+---------------------------------------------------------+
|5333265|{NULL, NULL, NULL, NULL, 1, NULL, NULL, NULL, NULL, NULL}|
|1333909|{NULL, NULL, NULL, 219, 5, NULL, NULL, NULL, NULL, NULL} |
|7327624|{NULL, NULL, 31, 8, NULL, 1, NULL, 1, NULL, NULL}        |
|6066819|{NUL

Since we're selecting the key in the struct with the highest value, we need to 0 the null values in the struct.

In [16]:
genreDF = genreDF.withColumn("nonull_genres", struct(
    when(col("genres.children").isNull(), 0).otherwise(col("genres.children")).alias("children"),
    when(col("genres.`comics, graphic`").isNull(), 0).otherwise(col("genres.`comics, graphic`")).alias("comics, graphic"),
    when(col("genres.`fantasy, paranormal`").isNull(), 0).otherwise(col("genres.`fantasy, paranormal`")).alias("fantasy, paranormal"),
    when(col("genres.fiction").isNull(), 0).otherwise(col("genres.fiction")).alias("fiction"),
    when(col("genres.`history, historical fiction, biography`").isNull(), 0).otherwise(col("genres.`history, historical fiction, biography`")).alias("history, historical fiction, biography"),
    when(col("genres.`mystery, thriller, crime`").isNull(), 0).otherwise(col("genres.`mystery, thriller, crime`")).alias("mystery, thriller, crime"),
    when(col("genres.`non-fiction`").isNull(), 0).otherwise(col("genres.`non-fiction`")).alias("non-fiction"),
    when(col("genres.poetry").isNull(), 0).otherwise(col("genres.poetry")).alias("poetry"),
    when(col("genres.romance").isNull(), 0).otherwise(col("genres.romance")).alias("romance"),
    when(col("genres.`young-adult`").isNull(), 0).otherwise(col("genres.`young-adult`")).alias("young-adult")
))
genreDF.show(10, truncate=False)

+--------+---------------------------------------------------------+----------------------------------+
|book_id |genres                                                   |nonull_genres                     |
+--------+---------------------------------------------------------+----------------------------------+
|5333265 |{NULL, NULL, NULL, NULL, 1, NULL, NULL, NULL, NULL, NULL}|{0, 0, 0, 0, 1, 0, 0, 0, 0, 0}    |
|1333909 |{NULL, NULL, NULL, 219, 5, NULL, NULL, NULL, NULL, NULL} |{0, 0, 0, 219, 5, 0, 0, 0, 0, 0}  |
|7327624 |{NULL, NULL, 31, 8, NULL, 1, NULL, 1, NULL, NULL}        |{0, 0, 31, 8, 0, 1, 0, 1, 0, 0}   |
|6066819 |{NULL, NULL, NULL, 555, NULL, 10, NULL, NULL, 23, NULL}  |{0, 0, 0, 555, 0, 10, 0, 0, 23, 0}|
|287140  |{NULL, NULL, NULL, NULL, NULL, NULL, 3, NULL, NULL, NULL}|{0, 0, 0, 0, 0, 0, 3, 0, 0, 0}    |
|287141  |{6, NULL, 1, 1, 9, NULL, NULL, NULL, NULL, 1}            |{6, 0, 1, 1, 9, 0, 0, 0, 0, 1}    |
|378460  |{NULL, NULL, NULL, 2, NULL, NULL, NULL, NULL, NULL, NU

In [42]:
genreDF = genreDF.drop(col('genres'))

To get the maximum key,value from the struct, we recreate dictionary representation of the struct as a workaround, get the max value, and return the key of that value.

In [43]:
def get_max_genre(genre_struct):
    # reconstruct the dictionary, beacuse genre_struct.jsonValue()
    # wasn't working in udf; pretty small so it shouldn't impact
    # performance
    children = genre_struct['children']
    comics = genre_struct['comics, graphic']
    fantasy = genre_struct['fantasy, paranormal']
    fiction = genre_struct.fiction
    history = genre_struct['history, historical fiction, biography']
    mystery = genre_struct['mystery, thriller, crime']
    non_fiction = genre_struct['non-fiction']
    poetry = genre_struct.poetry
    romance = genre_struct.romance
    young_adult = genre_struct['young-adult']

    # Create a dictionary from the extracted values
    genre_dict = {
        "children": children,
        "comics, graphic": comics,
        "fantasy, paranormal": fantasy,
        "fiction": fiction,
        "history, historical fiction, biography": history,
        "mystery, thriller, crime": mystery,
        "non-fiction": non_fiction,
        "poetry": poetry,
        "romance": romance,
        "young-adult": young_adult
    }
    return max(genre_dict, key=genre_dict.get)

get_max_genres_udf = udf(get_max_genre, StringType())
genre_df = genreDF.withColumn('primary_genre', get_max_genres_udf(col('nonull_genres')))
genre_df = genre_df.drop('nonull_genres')
genre_df.show(15, truncate=False)

+--------+--------------------------------------+
|book_id |primary_genre                         |
+--------+--------------------------------------+
|5333265 |history, historical fiction, biography|
|1333909 |fiction                               |
|7327624 |fantasy, paranormal                   |
|6066819 |fiction                               |
|287140  |non-fiction                           |
|287141  |history, historical fiction, biography|
|378460  |fiction                               |
|6066812 |fantasy, paranormal                   |
|34883016|romance                               |
|287149  |non-fiction                           |
|6066814 |history, historical fiction, biography|
|33394837|mystery, thriller, crime              |
|89371   |non-fiction                           |
|28575155|children                              |
|89373   |fiction                               |
+--------+--------------------------------------+
only showing top 15 rows



Now we can join the genre dataframe onto the main book dataframe.

In [19]:
bookGenreDF = bookDFLanguages.join(genre_df, col('best_book_id') == col('book_id'))

<h3>Books</h3>

The books dataset has detailed information about the books themselves. It's similar to the 'works' dataset, but also has things like author, isbn, and a goodreads link.

In [22]:
pathB = "./Datasets/goodreads_books.json"
bookDetailDF = spark.read.json(pathB)
bookDetailDF.printSchema()

root
 |-- asin: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- link: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- popular_shelves: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- pub

We don't intend to use all of the information present, so we select just a subset of the fields.

In [23]:
bookDetailDF = bookDetailDF.select("book_id",\
                                   "work_id",\
                                   "isbn",\
                                   "isbn13",\
                                   "title",\
                                   "format",\
                                   "url",\
                                   "image_url",\
                                   "authors")

                                   #"description",\
                                   #"link",\

<h3>Author</h3>

The books dataset gives a list of authors. We just want the primary author, so we flatten it and just get the primary author's ID. Later in the notebook we'll load the authors dataset and join that in to also get the author's name. We wait to join in the authors dataset until we've filtered the bookDetailDF down to just the rows we plan to keep (i.e. the books that are in the bookGenreDF dataframe).

In [24]:
def get_author_id(authors_list):
    try:
        if authors_list and authors_list[0]:
            return int(authors_list[0]['author_id'])
        else:
            return 0
    except EOFError:
        return 0

get_author_id_udf = udf(get_author_id, IntegerType())

In [25]:
bookDetailDF = bookDetailDF.withColumn('author_id', get_author_id_udf(col('authors'))).drop('authors')

In [26]:
bookDetailDF = bookDetailDF.withColumnRenamed('book_id', 'det_book_id').withColumnRenamed('work_id', 'det_work_id')

In [27]:
bookDetailDF = bookDetailDF.filter(bookDetailDF.author_id != 0) # CAN REMOVE THIS

<h3>Joining it all</h3>
Join it all together: bookGenre has book, genre, and language information; bookDetailDF has book, author, and metadata information; authorDetailDF has author name information.

In [29]:
# now we have works+genres and books+author IDs, join them
fullBookDF = bookGenreDF.join(bookDetailDF, bookGenreDF.best_book_id == bookDetailDF.det_book_id)
#fullBookDF.count() # 1998

In [30]:
# join in author data to get the author name
pathA = "./Datasets/goodreads_book_authors.json"
authorDetailDF = spark.read.json(pathA)
#authorDetailDF.printSchema()

In [31]:
authorDetailDF = authorDetailDF.select(authorDetailDF.author_id, authorDetailDF.name).withColumnRenamed('author_id', 'a_author_id')

In [32]:
fullBookDF = fullBookDF.join(authorDetailDF, fullBookDF.author_id == authorDetailDF.a_author_id).drop('a_author_id')
fullBookDF = fullBookDF.withColumnRenamed('best_book_id', 'ITEM_ID')
#fullBookDF.count() # 1998

<h3>Write it</h3>

By default spark outputs one file per reducer. Because we're running this notebook locally and aren't using a distributed file system, there's no easy to way to coalesce the files back together after the fact. Even though it slows things down, there's enough memory to repartition into a single partition so that the full output is fed to a single file.

In [33]:
# repartition so we can write it to a single file
fullBookDF = fullBookDF.repartition(1)
fullBookDF.write.csv("personalize_full_book_data.csv", header=True, mode="overwrite")

In [44]:
fullBookDF.printSchema()

root
 |-- ITEM_ID: string (nullable = true)
 |-- original_publication_day: string (nullable = true)
 |-- original_publication_month: string (nullable = true)
 |-- original_publication_year: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- rating_dist: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- ratings_sum: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- work_id: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- inferred_language_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- primary_genre: string (nullable = true)
 |-- det_book_id: string (nullable = true)
 |-- det_work_id: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- title: string (nullable = true)
 |-- format: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nu

<h3>Interactions</h3>

Now we load the interactions data. We only want to keep user:item interactions that were with an item that we haven't filtered out. We collect the bookIDs into a list, and filter the interactions df on that list.

In [35]:
# get the IDs before we re-partition and write
# used later to filter the interactions dataset
bookIDs = fullBookDF.select("ITEM_ID").rdd.flatMap(lambda x: x).collect()

In [36]:
# use the bookIDs to truncate / filter down the interactions dataset which is MASSIVE
pathI = "./Datasets/goodreads_interactions.csv"
interactionsDF = spark.read.option("header", True).csv(pathI)

In [37]:
interactionsDF.printSchema()
#interactionsDF.count() # prefiltered count: 228,648,342

root
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- is_read: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- is_reviewed: string (nullable = true)



In [38]:
# bookDetailDF = bookDetailDF.filter(bookDetailDF.author_id != 0)
interactionsDF = interactionsDF.filter(interactionsDF.is_read == 1) # 112,131,203
interactionsDF = interactionsDF.filter(interactionsDF.book_id.isin(bookIDs)) # 1,381,758
#interactionsDF.count() # post filtered count 1,381,758

1381758

Again, we repartition into a single partition so that we're writing out the contents to a single CSV file.

In [41]:
interactionsDF = interactionsDF.repartition(1)
interactionsDF.write.csv("interactions_data.csv", header=True, mode="overwrite")